In [21]:
import numpy as np
import pandas as df
import string
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
products=df.read_csv("C:/Users/gero_/OneDrive/Documents/python works/fuentes/amazon_baby.csv")

In [23]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
def remove_punctuation(text:str):
   import string
   for i in text:
      if i in string.punctuation:
         text=text.replace(i,'')
         
   return str(text)

In [25]:
products['review_clean']=products['review'].apply(lambda x :remove_punctuation(str(x)))

In [57]:
products.loc[10551]

name                                                          NaN
review          If you're searching for an elegant, inexpensiv...
rating                                                          4
review_clean    If youre searching for an elegant inexpensive ...
sentiment                                                       1
Name: 10551, dtype: object

In [26]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

# Extracción de Sentimientos

In [27]:
products=products[products['rating']!=3]

In [28]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [29]:
products[products['rating']==2]

,name,review,rating,review_clean,sentiment
21,Nature's Lullabies Second Year Sticker Calendar,I only purchased a second-year calendar for my...,2,I only purchased a secondyear calendar for my ...,-1
41,"SoftPlay Giggle Jiggle Funbook, Happy Bear",This bear is absolutely adorable and I would g...,2,This bear is absolutely adorable and I would g...,-1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
78,Cloth Diaper Pins Stainless Steel Traditional ...,These were good quality--worked fine--heavy d...,2,These were good qualityworked fineheavy dutya...,-1
80,Cloth Diaper Pins Stainless Steel Traditional ...,"While the diaper pins are attractive, the meta...",2,While the diaper pins are attractive the metal...,-1
...,...,...,...,...,...
183293,Aqueduck Faucet Extender &amp; Handle Extended...,I wish I had bought the faucet extender and ha...,2,I wish I had bought the faucet extender and ha...,-1
183302,Vandot 2 in1 Accessory Set 3D Leather Case Lit...,Cute but cheaply made.. The part where you put...,2,Cute but cheaply made The part where you put y...,-1
183371,Summer Infant Pop 'n Play Portable Playard,Good idea but too dangerous. I really wanted t...,2,Good idea but too dangerous I really wanted to...,-1
183472,Bouncy&reg; Inflatable Real Feel Hopping Cow,When I received the item my initial thought wa...,2,When I received the item my initial thought wa...,-1


In [30]:
import json

# Leer el archivo JSON con los índices
with open('C:/Users/gero_/OneDrive/Documents/python works/fuentes/module-2-assignment-train-idx.json', 'r') as file:
    indices_train = json.load(file)
file.close()


In [31]:
with open('C:/Users/gero_/OneDrive/Documents/python works/fuentes/module-2-assignment-test-idx.json', 'r') as file2:
    indices_test = json.load(file2)
file2.close()


In [32]:
train_data=products.iloc[indices_train]
test_data=products.iloc[indices_test]

In [33]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [34]:
sentiment_model=LogisticRegression(max_iter=1000)
sentiment_model.fit(train_matrix,train_data['sentiment'])

LogisticRegression(max_iter=1000)

In [35]:
np.size(sentiment_model.coef_)

121713

In [36]:
count1=0 #para positivos
count2=0 #para negativos
for i in sentiment_model.coef_[0]:
         if i>=0:
          count1=count1+1
         else:
          count2=count2+1
print(count1,count2)

90931 30782


In [37]:
sample_test_data = test_data[10:13]
sample_test_data

,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


In [39]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix) #gets the score function

In [40]:
vectorizer.get_feature_names_out()

array(['0', '00', '000', ..., 'zzzzzz', 'zzzzzzz', 'zzzzzzzzzzz'],
      dtype=object)

In [41]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1])

In [42]:
def cal_probability (score:np.array):
    return 1/( 1+ np.exp( -score ) )

In [43]:
cal_probability(scores)

array([9.96381181e-01, 3.86692363e-02, 2.73677697e-05])

In [44]:
test_data['probablity']=cal_probability(sentiment_model.decision_function(test_matrix))

C:\Users\gero_\AppData\Local\Temp\ipykernel_7812\3485718050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['probablity']=cal_probability(sentiment_model.decision_function(test_matrix))


In [45]:
test_data.sort_values('probablity',ascending=False).head(20)

,name,review,rating,review_clean,sentiment,probablity
66059,"Evenflo 6 Pack Classic Glass Bottle, 4-Ounce",It's always fun to write a review on those pro...,5,Its always fun to write a review on those prod...,1,1.0
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,1,1.0
180646,Mamas &amp; Papas 2014 Urbo2 Stroller - Black,After much research I purchased an Urbo2. It's...,4,After much research I purchased an Urbo2 Its e...,1,1.0
114796,"Fisher-Price Cradle 'N Swing, My Little Snuga...",My husband and I cannot state enough how much ...,5,My husband and I cannot state enough how much ...,1,1.0
165593,Ikea 36 Pcs Kalas Kids Plastic BPA Free Flatwa...,For the price this set is unbelievable- and tr...,5,For the price this set is unbelievable and tru...,1,1.0
80155,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,1,1.0
133651,"Britax 2012 B-Agile Stroller, Red",[I got this stroller for my daughter prior to ...,4,I got this stroller for my daughter prior to t...,1,1.0
97325,Freemie Hands-Free Concealable Breast Pump Col...,I absolutely love this product. I work as a C...,5,I absolutely love this product I work as a Cu...,1,1.0
147949,"Baby Jogger City Mini GT Single Stroller, Shad...","Amazing, Love, Love, Love it !!! All 5 STARS a...",5,Amazing Love Love Love it All 5 STARS all the...,1,1.0
50315,"P'Kolino Silly Soft Seating in Tias, Green",I've purchased both the P'Kolino Little Reader...,4,Ive purchased both the PKolino Little Reader C...,1,1.0


In [46]:
test_data.sort_values('probablity',ascending=True).head(20)

,name,review,rating,review_clean,sentiment,probablity
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1,5.071675e-16
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1,1.858279e-15
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,-1,5.102891e-14
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1,1.113968e-13
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1,1.474925e-13
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1,4.138264e-13
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1,2.671635e-11
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1,3.005521e-11
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,-1,7.192507e-11
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,-1,8.377304e-11


In [47]:
def accuracy_cal(predicciones,target):
    cont=0
    for i,j in zip(predicciones,target):
        if i==j:
            cont=cont+1
    print (cont/len(predicciones))

In [51]:
accuracy_cal(sentiment_model.predict(test_matrix),np.reshape((test_data[['sentiment']]),-1))

0.9324754019678426


## Otro clasificador

In [59]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [60]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [64]:
vectorizer_word_subset.get_feature_names_out()

array(['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves',
       'well', 'able', 'car', 'broke', 'less', 'even', 'waste',
       'disappointed', 'work', 'product', 'money', 'would', 'return'],
      dtype=object)

In [65]:
train_matrix_word_subset.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 1, 0]])

In [66]:
simple_model=LogisticRegression(max_iter=1000)
simple_model.fit(train_matrix_word_subset,train_data['sentiment'])

LogisticRegression(max_iter=1000)

In [71]:
np.reshape(simple_model.coef_,-1)

array([ 1.36527333,  0.94879994,  1.19607997,  0.08749949,  0.52301419,
        1.50232359,  1.65529344,  0.50370774,  0.1871668 ,  0.05741787,
       -1.65820173, -0.20052515, -0.51172751, -1.99230625, -2.35884432,
       -0.62070233, -0.32265943, -0.9043337 , -0.36255349, -2.12320027])

In [72]:
words_coef= dict()
for ii,jj in zip(significant_words,np.reshape(simple_model.coef_,-1)):
    words_coef[ii]=jj

In [80]:
sorted(words_coef.items(), key=lambda item: item[1])

[('disappointed', np.float64(-2.358844318335462)),
 ('return', np.float64(-2.123200268546418)),
 ('waste', np.float64(-1.992306249503908)),
 ('broke', np.float64(-1.6582017283886588)),
 ('money', np.float64(-0.9043337025500089)),
 ('work', np.float64(-0.6207023254032783)),
 ('even', np.float64(-0.5117275050808044)),
 ('would', np.float64(-0.36255349246028223)),
 ('product', np.float64(-0.3226594287615908)),
 ('less', np.float64(-0.20052515420818984)),
 ('car', np.float64(0.05741786589579973)),
 ('old', np.float64(0.0874994889220263)),
 ('able', np.float64(0.18716680094363283)),
 ('well', np.float64(0.503707744741021)),
 ('little', np.float64(0.5230141857947209)),
 ('great', np.float64(0.9487999378571466)),
 ('easy', np.float64(1.196079971250815)),
 ('love', np.float64(1.365273325586228)),
 ('perfect', np.float64(1.5023235891348667)),
 ('loves', np.float64(1.6552934437928588))]

## Comparando modelos 

para el conjunto de datos train

In [82]:
accuracy_cal(sentiment_model.predict(train_matrix),np.reshape((train_data[['sentiment']]),-1))

0.968497031840259


In [87]:
accuracy_cal(simple_model.predict(train_matrix_word_subset ),np.reshape((train_data[['sentiment']]),-1))

0.8668000839479523


para el conjunto de datos test

In [88]:
accuracy_cal(sentiment_model.predict(test_matrix),np.reshape((test_data[['sentiment']]),-1))

0.9324754019678426


In [89]:
accuracy_cal(simple_model.predict(test_matrix_word_subset ),np.reshape((test_data[['sentiment']]),-1))

0.8695704343652508


## Prediccion de la clase mayoritaria 

In [96]:
cont3=0
cont4=0
for i in test_data['sentiment']:
    if i>=0:
        cont3=cont3+1
    else:
        cont4=cont4+1
if cont3>cont4:
    print ('clase mayoritaria +1 con : '+str(cont3)+' registros')
else:
    print('clase mayoritaria -1 con : '+str(cont4)+' registros')


clase mayoritaria +1 con : 28095 registros


In [104]:
accuracy_cal(np.full(len(test_data['sentiment']),1),np.reshape((test_data[['sentiment']]),-1))

0.8427825773938085
